In [1]:
###### 파일준비 
### 불용어 , 끝 

In [1]:
!apt-get update > /dev/null
!apt-get install g++ openjdk-8-jdk > /dev/null
!pip install konlpy > /dev/null

In [ ]:
### okt안되는부분 해결


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

In [3]:
from konlpy.tag import Okt

In [4]:
okt=Okt()

In [5]:
def test2(s):
  hangul = re.compile('[^ 0-9+.]')
  result = hangul.sub('', s) 
  return (result)

In [6]:
## recommend함수에 필요
kw = pd.read_csv('한글불용어100.txt', sep='\s+', header=None)
stopwords = set(kw[0])

In [7]:
## 데이터 불러오고 null값 제거후 index정리
df = pd.read_csv('끝.csv',lineterminator='\n')
df = df[df['영화명'].notna()].reset_index()
## 불필요한 열
del df['index']
del df['Unnamed: 0']

###### 개봉일 날짜에서 월에 해당하는 부분만 꺼내 type을 정수로 변환시키고 계절로 분류시켰습니다
##### 나중에 더미변수로 활용 기대
month_list=[]
for i in range(len(df.개봉일)):
  try:
    if int(df.개봉일[i][-11:-7]) == 19 or int(df.개봉일[i][-11:-7]) == 20:
      if 3<=int(df.개봉일[i][-5:-3])<6:
        month_list.append('봄')
      elif 6<=int(df.개봉일[i][-5:-3])<9:
        month_list.append('여름')
      elif 9<=int(df.개봉일[i][-5:-3])<12:
        month_list.append('가을')
      else:
        month_list.append('겨울')
    else:
      if 3<=int(df.개봉일[i][-7:-5])<6:
        month_list.append('봄')
      elif 6<=int(df.개봉일[i][-7:-5])<9:
        month_list.append('여름')
      elif 9<=int(df.개봉일[i][-7:-5])<12:
        month_list.append('가을')
      else:
        month_list.append('겨울')
  except:
    month_list.append(None)

df['계절']=month_list

### 개봉년도를 column에 추가해 보겠습니다
year_list=[]
for i in range(len(df.개봉일)):
  try:
    x = int(df.개봉일[i][-11:-7])
    if x==19 or x == 20:
      x=df.개봉일[i][-9:-5]
    year_list.append(int(x))
  except:
    year_list.append(None)

df['개봉연도']=year_list
####
##### 추천 열에 해당하는 텍스트중에 필요한 정보들만 뽑아 새로운 column으로 만들겠습니다.
### 추천 텍스트에서 연령층
per_list=[]
for i in range(len(df.개봉일)):
  try:
    x = df['추천'][i][6:8]
    per_list.append(int(x))
  except:
    per_list.append(None)
df['추천_연령대']=per_list

### 추천 텍스트에서 성별
sex_list=[]
for i in range(len(df.개봉일)):
  try:
    x = df['추천'][i][10:12]
    sex_list.append(x)
  except:
    sex_list.append(None)
df['추천_성별']=sex_list

### 추천 텍스트에서 특징
spe_list=[]
for i in range(len(df.개봉일)):
  try:
    x = df['추천'][i][19:-12]
    spe_list.append(x)
  except:
    spe_list.append(None)
df['영화_특징']=spe_list

#### 불필요한 
del df['추천']

# 연령비율 형태 변경 '%'없애고 실수형으로 표시

a1_list=[]
a2_list=[]
a3_list=[]
a4_list=[]
a5_list=[]
for i in range(len(df['영화명'])):
    try:
      a1 = float(df['10대 비율'][i].replace('%',''))
    except:
      a1 = None
    try:
      a2 = float(df['20대 비율'][i].replace('%',''))
    except:
      a2 = None
    try:
      a3 = float(df['30대 비율'][i].replace('%',''))
    except:
      a3 = None
    try:
      a4 = float(df['40대 비율'][i].replace('%',''))
    except:
      a4 = None
    try:
      a5 = float(df['50대 비율'][i].replace('%',''))
    except:
      a5 = None
    a1_list.append(a1)
    a2_list.append(a2)
    a3_list.append(a3)
    a4_list.append(a4)
    a5_list.append(a5)
df['10대 비율']=a1_list
df['20대 비율']=a2_list
df['30대 비율']=a3_list
df['40대 비율']=a4_list
df['50대 비율']=a5_list


#### 남성 여성

b1_list=[]
b2_list=[]

for i in range(len(df['영화명'])):
    try:
      b1 = float(df['남성 만족도'][i])
    except:
      b1 = None
    try:
      b2 = float(df['여성 만족도'][i])
    except:
      b2 = None

    b1_list.append(b1)
    b2_list.append(b2)

df['남성 만족도']=b1_list
df['여성 만족도']=b2_list

##관람객 평점
##### 점수가 없는 것들은 ' ' 으로 되어있음
c1_list=[]

for i in range(len(df['영화명'])):
    try:
      c1 = float(test2(df['관람객평점'][i]))
    except:
      c1 = None
    c1_list.append(c1)
df['관람객평점']=c1_list

d1_list=[]

for i in range(len(df['영화명'])):
    try:
      d1 = float(df['관객평점인원'][i][3:-1])
    except:
      d1 = None
    d1_list.append(d1)
df['관객평점인원']=d1_list

e1_list=[]
for i in range(len(df['영화명'])):
    try:
      e1 = float(df['네티즌평점인원'][i][3:-1])
    except:
      e1 = None
    e1_list.append(e1)
df['네티즌평점인원']=e1_list
## 관객수 float화


In [32]:
def recommend(x1,x2,x3,x4,x5):
  
  df['스토리'] = df['스토리'].str.replace('\r','')
  cleanr = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
  df['블로그'] = df['블로그'].replace(cleanr,'')
  df['블로그'].iloc[0]
  dff = df.sort_values('순위').head(30)
  dff
  dffind = pd.Series(dff.index, index=dff['영화명'])
  ind=dffind[x5]
  new_total =df.loc[ind]['스토리']+df.loc[ind]['블로그']      ###### 고른 영화 total 데이터
  
  mv_jang=df.drop(index=[231,388])
  mv_jang = mv_jang.reset_index()
  del mv_jang['index']
  ## 장르부분을 리스트화
  jang_list=[]
  for i in range(len(mv_jang['장르'])):
    j = mv_jang['장르'][i].split()
    jang_list.append(j)
  mv_jang['장르'] = jang_list
  j_list=[]
  ##네이버 영화에 장르가 몇종류 있는지 꺼내왔습니다.
  for i in range(len(mv_jang['장르'])):
    for j in range(5):
      try:
        j = mv_jang['장르'][i][j]
        if j in j_list:
          pass
        else:
          j_list.append(j)
      except:
        pass
  ## 더미변수 화
  mv_j=mv_jang
  for i in j_list:
    a=[]
    for k in range(len(mv_jang['장르'])):
      if i in mv_jang['장르'][k]:
        b=1
      else:
        b=0
      a.append(b)
    mv_j[i]=a
  ##영화특징 더미변수화
  sang_list=[]
  for i in range(len(mv_j['장르'])):
    try:
      j = mv_j['영화_특징'][i].split(', ')
    except:
      j = ''
    sang_list.append(j)
  mv_j['영화_특징'] = sang_list
  s_list=[]
  ##영화 특징이 몇종류 있는지 꺼내왔습니다.
  for i in range(len(mv_j['장르'])):
    for j in range(5):
      try:
        j = mv_j['영화_특징'][i][j]
        if j in s_list:
          pass
        else:
          s_list.append(j)
      except:
        pass
  mv_s=mv_j
  for i in s_list:
    a=[]
    for k in range(len(mv_s['영화_특징'])):
      if i in mv_s['영화_특징'][k]:
        b=1
      else:
        b=0
      a.append(b)
    mv_s[f'특:{i}']=a
  del mv_s['특:']
  jjjj = pd.DataFrame(j_list)
  x_4=jjjj.iloc[x4][0]
  ssss = pd.DataFrame(s_list)
  x_3=ssss.iloc[x3][0]
  a_ = mv_s[mv_s['추천_연령대']==x1] 
  b_ = a_[a_['추천_성별']==x2]
  c_ = b_[b_[f'특:{x_3}']==1]
  d_ = c_[c_[x_4]==1]
  d_ = d_[d_['스토리'].notna()]
  d_ = d_[d_['블로그'].notna()]
  d_ = d_.reset_index()
  del d_['index']
  d_['total'] = d_['스토리'] + d_['블로그']
  ddl = []
  for s in d_['total']:
    morphs = okt.morphs(s, stem=True)
    temp_str = ' '.join(word for word in morphs if word not in stopwords)
    ddl.append(temp_str)
  d_['total']=ddl
  d2 = pd.DataFrame(d_['total'])
  d2 = d2.append({'total':new_total},ignore_index=True)
  d2['total'] ### len(d2)-1 추가된 데이터 인덱스
  tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0)
  tfidf_matrix = tf.fit_transform(d2['total'])
  cosine_sim = linear_kernel(tfidf_matrix,tfidf_matrix)
  indices = pd.Series(d_.index, index=d_['영화명'])
  idx = len(d2)-1 # 영화명에 해당하는 인덱스
  sim_scores = list(enumerate(cosine_sim[idx])) # 해당 인덱스의 cosine_sim 추출
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True) # 추출한 cosine_sim을 정렬
  sim_scores = sim_scores[1:4] # 상위 30개만 추출
  movie_indices = [i[0] for i in sim_scores]
  return d_.iloc[movie_indices][['영화명','네티즌평점','개봉일','감독','장르','배우']]

In [ ]:
## x1 = 연령 , x2 = 성별 , x3 = 특징, x4 = 장르, x5= 영화번호

In [33]:
recommend(20,'남성',3,9,0)

,영화명,네티즌평점,개봉일,감독,장르,배우
42,무사,8.85,20010907 개봉,김성수,"[드라마, 액션]","['안성기', '정우성', '주진모', '장쯔이', '우영광', '이두일', '박용..."
93,데몰리션 맨,8.37,19931120 개봉,마르코 브람빌라,[액션],"['실베스터 스탤론', '웨슬리 스나입스', '산드라 블록', '나이젤 호손', '..."
38,쉬리,8.89,19990213 개봉,강제규,"[액션, 멜로로맨스, 드라마]","['한석규', '최민식', '송강호', '김윤진', '윤주상', '박용우', '박하..."


In [8]:
x = df.sort_values('순위')['영화명'].head(30).tolist()
x_ =pd.DataFrame(x)
x_


,0
0,명량
1,극한직업
2,신과함께-죄와 벌
3,국제시장
4,어벤져스: 엔드게임
5,겨울왕국 2
6,아바타
7,베테랑
8,괴물
9,도둑들


In [44]:
jjjj

,0
0,다큐멘터리
1,드라마
2,애니메이션
3,모험
4,코미디
5,전쟁
6,멜로로맨스
7,공연실황
8,판타지
9,액션


In [46]:
ssss

,0
0,스토리
1,연기
2,영상미
3,연출
4,OST
5,
6,모든면에


In [20]:
x1=20
x2='여성'
x3=3
x4=3
x5=0

In [9]:
df['스토리'] = df['스토리'].str.replace('\r','')
cleanr = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
df['블로그'] = df['블로그'].replace(cleanr,'')
df['블로그'].iloc[0]
dff = df.sort_values('순위').head(30)
dff
dffind = pd.Series(dff.index, index=dff['영화명'])
dffind[x5]
ind=dffind[1]
new_total =df.loc[ind]['스토리']+df.loc[ind]['블로그']      ###### 고른 영화 total 데이터

mv_jang=df.drop(index=[231,388])
mv_jang = mv_jang.reset_index()
del mv_jang['index']
## 장르부분을 리스트화
jang_list=[]
for i in range(len(mv_jang['장르'])):
  j = mv_jang['장르'][i].split()
  jang_list.append(j)
mv_jang['장르'] = jang_list
j_list=[]
##네이버 영화에 장르가 몇종류 있는지 꺼내왔습니다.
for i in range(len(mv_jang['장르'])):
  for j in range(5):
    try:
      j = mv_jang['장르'][i][j]
      if j in j_list:
        pass
      else:
        j_list.append(j)
    except:
      pass
## 더미변수 화
mv_j=mv_jang
for i in j_list:
  a=[]
  for k in range(len(mv_jang['장르'])):
    if i in mv_jang['장르'][k]:
      b=1
    else:
      b=0
    a.append(b)
  mv_j[i]=a
##영화특징 더미변수화
sang_list=[]
for i in range(len(mv_j['장르'])):
  try:
    j = mv_j['영화_특징'][i].split(', ')
  except:
    j = ''
  sang_list.append(j)
mv_j['영화_특징'] = sang_list
s_list=[]
##영화 특징이 몇종류 있는지 꺼내왔습니다.
for i in range(len(mv_j['장르'])):
  for j in range(5):
    try:
      j = mv_j['영화_특징'][i][j]
      if j in s_list:
        pass
      else:
        s_list.append(j)
    except:
      pass
mv_s=mv_j
for i in s_list:
  a=[]
  for k in range(len(mv_s['영화_특징'])):
    if i in mv_s['영화_특징'][k]:
      b=1
    else:
      b=0
    a.append(b)
  mv_s[f'특:{i}']=a
del mv_s['특:']
jjjj = pd.DataFrame(j_list)
x_4=jjjj.iloc[x4][0]
ssss = pd.DataFrame(s_list)
x_3=ssss.iloc[x3][0]


NameError: ignored